In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from itertools import product
from competition.titanic.FeaturesEngineering import *
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("error", category=FutureWarning)

from sklearn.pipeline import Pipeline

(891, 11)
(891,)


In [2]:
df = pd.read_csv("../data/train.csv", delimiter=",")

features = [x for x in df.columns if x != "Survived"]
x, y = df[features], df["Survived"].T
print(x.shape)
print(y.shape)

(891, 11)
(891,)


In [3]:
X_train, X_cv, y_train, y_cv = train_test_split(x, y, train_size=0.8, random_state=1)

In [4]:
max_depth_list = [2, 3, 4, 5, 6, 7, 8]
min_child_weight_list = [10, 20, 30, 40, 50]
n_estimators_list = [50, 100, 150, 200]
colsample_bytree_list = [0.6, 0.8, 1]


def eval_models(X_train, y_train, X_cv, y_cv,
                max_depth,
                min_child_weight,
                n_estimators,
                colsample_bytree,
                feature_transformer):
    model = Pipeline(steps=[
        ("preprocessor", feature_transformer),
        ("classifier", XGBClassifier(n_estimators=n_estimators,
                                     learning_rate=0.1,
                                     max_depth=max_depth,
                                     min_child_weight=min_child_weight,
                                     colsample_bytree=colsample_bytree,
                                     random_state=1))
    ])
    model.fit(X_train, y_train)
    y_cv_hat = model.predict(X_cv)
    cv_accuracy = np.mean(y_cv == y_cv_hat)
    print(
        f"Max depth: {max_depth}, n_estimators: {n_estimators}, min_child_weight: {min_child_weight}, colsample_bytree: {colsample_bytree}, cv_accuracy: {cv_accuracy}")
    return model, cv_accuracy


# for max_depth, min_child_weight, n_estimators, colsample_bytree in product(max_depth_list, min_child_weight_list, n_estimators_list, colsample_bytree_list):
#     eval_models(X_train, y_train, X_cv, y_cv, max_depth, min_child_weight, n_estimators, colsample_bytree, engineer_features())

best_model_params = max(
    [eval_models(X_train, y_train, X_cv, y_cv, max_depth, min_child_weight, n_estimators, colsample_bytree, engineer_features()) for max_depth, min_child_weight, n_estimators, colsample_bytree in
     product(max_depth_list, min_child_weight_list, n_estimators_list, colsample_bytree_list)], key=lambda x: x[1])
print(best_model_params[1])

Max depth: 2, n_estimators: 50, min_child_weight: 10, colsample_bytree: 0.6, cv_accuracy: 0.776536312849162
Max depth: 2, n_estimators: 50, min_child_weight: 10, colsample_bytree: 0.8, cv_accuracy: 0.776536312849162
Max depth: 2, n_estimators: 50, min_child_weight: 10, colsample_bytree: 1, cv_accuracy: 0.776536312849162
Max depth: 2, n_estimators: 100, min_child_weight: 10, colsample_bytree: 0.6, cv_accuracy: 0.776536312849162
Max depth: 2, n_estimators: 100, min_child_weight: 10, colsample_bytree: 0.8, cv_accuracy: 0.776536312849162
Max depth: 2, n_estimators: 100, min_child_weight: 10, colsample_bytree: 1, cv_accuracy: 0.776536312849162
Max depth: 2, n_estimators: 150, min_child_weight: 10, colsample_bytree: 0.6, cv_accuracy: 0.776536312849162
Max depth: 2, n_estimators: 150, min_child_weight: 10, colsample_bytree: 0.8, cv_accuracy: 0.776536312849162
Max depth: 2, n_estimators: 150, min_child_weight: 10, colsample_bytree: 1, cv_accuracy: 0.776536312849162
Max depth: 2, n_estimators: 

In [ ]:
df_test = pd.read_csv("../data/test.csv", delimiter=",")

best_model = best_model_params[0]
y_test_hat = best_model.predict(df_test)
output = pd.DataFrame({
    "PassengerId": df_test["PassengerId"],
    "Survived": y_test_hat
})
output.to_csv("submission.csv", index=False)